In [1]:
using Revise
using LinearAlgebra
using AdaptiveTransportMap
using Statistics
using Distributions
using Test
using ForwardDiff
using SpecialFunctions
using OrthoMatchingPursuit
using BenchmarkTools

In [21]:
Nx = 5
Ne = 500
X = randn(Nx, Ne).^2
Xvalid = randn(Nx, Ne).^2
λ = 0.0
δ = 1e-8
γ = 2.0

2.0

In [22]:
@time Cgreedy, ϵ = greedyfit(5, 2, 0, X, 1, λ, δ, γ; verbose = true)

1 active dimensions  - Training error: 0.8027217746040775
2 active dimensions  - Training error: 0.8002350736658196
  0.845906 seconds (1.43 M allocations: 63.141 MiB, 1.55% gc time)


(Sparse radial map component of dimension 5 and order p = [-1, 2, -1, -1, 0]
, [0.8027217746040775, 0.8002350736658196])

In [25]:
Cgreedy = SparseRadialMapComponent(Nx, -1)
@time Cgreedy, ϵ = optimize(Cgreedy, X, 2, 0, "kfold", λ, δ, γ; verbose = true)

Fold 1:
1 active dimensions  - Training error: 0.7289785449152474, Validation error: 1.12891236410568
2 active dimensions  - Training error: 0.7277546847781257, Validation error: 1.1244317773371784
3 active dimensions  - Training error: 0.7265975022157747, Validation error: 1.1224628742102827
4 active dimensions  - Training error: 0.7254093325451157, Validation error: 1.1284116685665175
5 active dimensions  - Training error: 0.7245662133352425, Validation error: 1.1347274827088785
Fold 2:
1 active dimensions  - Training error: 0.8493718279725815, Validation error: 0.6276677117864736
2 active dimensions  - Training error: 0.8474511403120049, Validation error: 0.6262746840124936
3 active dimensions  - Training error: 0.846034620609899, Validation error: 0.6266013699660019
4 active dimensions  - Training error: 0.8452712264106544, Validation error: 0.6268295216719451
5 active dimensions  - Training error: 0.8448849365721562, Validation error: 0.6321241394630992
Fold 3:
1 active dimensions

(Sparse radial map component of dimension 5 and order p = [-1, -1, -1, -1, 0]
, [0.8027217746040775])

In [ ]:
@time Cgreedy, ϵ = optimize(Cgreedy, X, 2, 0, "split", λ, δ, γ; maxpatience = 4)

In [19]:
@time Cgreedy, error = greedyfit(Nx, 2, 2, deepcopy(X), 3, λ, δ, γ)

  0.018648 seconds (91.66 k allocations: 11.540 MiB)


(Sparse radial map component of dimension 5 and order p = [2, -1, 2, 2, 2]
, [0.16149625717495586, 0.15955843305309023, 0.1513458062113992, 0.14608833904374321])

In [20]:
@time Cgreedy, error = greedyfit(Nx, 2, 0, deepcopy(X), deepcopy(Xvalid), 4, λ, δ, γ; verbose = false)

  0.004808 seconds (54.84 k allocations: 5.174 MiB)


(Sparse radial map component of dimension 5 and order p = [2, 2, 2, 2, 0]
, ([0.9522111470540667, 0.9468735067392048, 0.9382460125763467, 0.9339206542025913, 0.9330111942049354], [0.8123465391957869, 0.8138996919068819, 0.8178551591012666, 0.8222529910064743, 0.8217679113283066]))

In [33]:
C = SparseRadialMapComponent(Nx, Cgreedy.p)

Sparse radial map component of dimension 5 and order p = [1, 1, 1, 1, 0]


In [34]:
@time center_std!(C, sort(deepcopy(X); dims = 2); γ = γ)

  0.007385 seconds (249 allocations: 114.234 KiB)


In [35]:
@time x_opt = optimize(C, X, λ, δ)
modify_a!(C, x_opt)

  0.000287 seconds (2.58 k allocations: 277.781 KiB)


In [36]:
negative_likelihood(C, X)

0.9143832086452744

In [37]:
negative_likelihood(Cgreedy, X)

0.9143832086452746